# Réseau de neurones et attrition des clients bancaires

Réseau de neurones avec Python (Keras)

## La problématique

On s'intéresse ici à une problème classique du domaine bancaire (mais pas que !) qui est l'attrition ou *churn* en anglais et correspond à la perte de client. 
Récemment de nombreux clients ont quitté la banque Crédit Friqué. La question est de comprendre pourquoi ces départs ?

## Les données

Pour répondre à cette question, la banque a sélectionné 6 mois plus tôt un sous ensemble de ses clients pour lesquels elle a stocké un certain nombre d’information puis, dans les 6 mois qui ont suivis, elle a observé si les clients avaient quitté ou non la banque. Vous voilà donc, 6 mois plus tard, contacté par la banque qui vous propose un beau dataset (pour une fois!) et vous demande de déterminer les profils des clients les plus à même de partir.
Vous disposez du fichier banque_abandon.csv qui est la base de données de la banque virtuelle Crédit Friqué.

## Quelques questions préliminaires

C'est juste pour vous échauffer donc ça doit être fait en moins d'une heure ça !
1. À quoi correspondent les différentes variables du datasets ?
2. Pour pas perdre les bonnes habitudes, faites quelques visualisations pour voir ce qu'il y a dans vos données.
3. À quelle type de problème avez-vous à faire ici : classification ou régression ?
>- Lister un certain nombre de modèles vous permettant de le résoudre
>- Lister les métriques associées à ce type de problème
>- Choisir un modèle, l'entraîner et l'évaluer avec la métrique de votre choix

In [ ]:
# Alors là, flemme de refaire et puis vous êtes censés être au point là-dessus depuis le temps...si y a des questions, évidemment, posez-les

## Dans le vif du sujet

Vous l'aurez compris, il s'agit ici de résoudre le problème à l'aide d'un réseau de neurones.   
Vous aurez bien sûr besoin du package `keras` et il vous faudra aussi certainement installer `tensorflow`(et peut-être `theano` si besoin).  
À vous de jouer !
N'oubliez pas le preprocessing !

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Dataset
df = pd.read_csv('data/banque_abandon.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Création des tables X et y
X = df.iloc[:, 3:13] #pour pas prendre les variable id, name et label
y = df.iloc[:, 13] #juste le label
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [4]:
X.value_counts('Geography')

Geography
France     5014
Germany    2509
Spain      2477
dtype: int64

In [5]:
# One-hot-encoding sur variables catégoriques
# et feature scaling sur variables numériques
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
preprocess = make_column_transformer(
        (OneHotEncoder(), ['Geography', 'Gender']),
        (StandardScaler(), ['CreditScore', 'Age', 'Tenure', 'Balance',
                            'NumOfProducts', 'HasCrCard', 'IsActiveMember', 
                            'EstimatedSalary']))
X = preprocess.fit_transform(X)

In [6]:
# Échantillons test et train
#y = y.values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
from keras.models import Sequential
from keras.layers import Dense

# Architecture
mlp_clf = Sequential()
mlp_clf.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 13))
mlp_clf.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
mlp_clf.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Paramètres d'apprentissage
mlp_clf.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Entraînement
mlp_clf.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 6s 7ms/step - loss: 0.4783 - accuracy: 0.7958
Epoch 2/100
800/800 [==============================] - 5s 6ms/step - loss: 0.4273 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 4s 5ms/step - loss: 0.4228 - accuracy: 0.8089
Epoch 4/100
800/800 [==============================] - 5s 6ms/step - loss: 0.4194 - accuracy: 0.8230
Epoch 5/100
800/800 [==============================] - 5s 6ms/step - loss: 0.4169 - accuracy: 0.8266
Epoch 6/100
800/800 [==============================] - 5s 6ms/step - loss: 0.4148 - accuracy: 0.8294
Epoch 7/100
800/800 [==============================] - 5s 6ms/step - loss: 0.4132 - accuracy: 0.8305
Epoch 8/100
800/800 [==============================] - 5s 6ms/step - loss: 0.4119 - accuracy: 0.8319
Epoch 9/100
800/800 [==============================] - 5s 6ms/step - loss: 0.4107 - accuracy: 0.8320
Epoch 10/100
800/800 [==============================] - 6s 7ms/step - loss: 0.4095 - accura

In [8]:
# Accuracy
mlp_clf.evaluate(X_test, y_test)

63/63 [==============================] - 0s 4ms/step - loss: 0.3959 - accuracy: 0.8405


[0.39585408568382263, 0.840499997138977]

In [9]:
# Prédiction sur le test set
y_pred = mlp_clf.predict(X_test)
y_pred = 1*(y_pred > 0.5)

In [11]:
# Matrice de confusion
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1549,   46],
       [ 273,  132]])

In [12]:
# Prédiction pour un nouveau client
"""On veut prédire si le client suivant va quitter la banque:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
Xnew = pd.DataFrame(data={
        'CreditScore': [600], 
        'Geography': ['France'], 
        'Gender': ['Male'],
        'Age': [40],
        'Tenure': [3],
        'Balance': [60000],
        'NumOfProducts': [2],
        'HasCrCard': [1],
        'IsActiveMember': [1],
        'EstimatedSalary': [50000]})
Xnew = preprocess.transform(Xnew)
new_prediction = mlp_clf.predict(Xnew)
new_prediction, (new_prediction > 0.5)

(array([[0.09119666]], dtype=float32), array([[False]]))

## Évaluation du réseau et affinage des hyper-paramètres

Jusqu'à maintenant, on a évalué les réseaux qu'on a vu en regardant uniquement l'accuracy mais cette valeur n'est pas déterministe puisqu'elle dépend de certains paramètres aléatoires comme le train_test_split, l'intialisation des paramètres etc...

Une solution par rapport à ce problème est de répéter l'entraînement plusieurs fois et de regarder les résultats en moyenne. On l'a déjà utilisé et ça s'appelle la validation croisée.

Mettez en place la validation croisée en utilisant `cross_val_score` puis affiner les paramètres avec `GridSearchCV`.

**/!\\** Vous aurez besoin de ce qu'on appelle un wrapper pour pouvoir relier `keras` à `sklearn` et utiliser un modèle de l'un dans l'autre. Ça tombe bien, ça existe : regarder la librairie `keras.wrappers.scikit_learn`.

In [13]:
# Définition de la fonction de création du modèle
def build_classifier(optimizer='adam'):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 13))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [14]:
# Validation croisée avec cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, 
                             cv = 10, n_jobs = -1)

In [15]:
accuracies

array([0.85874999, 0.85250002, 0.88      , 0.85124999, 0.86750001,
       0.82999998, 0.86500001, 0.82249999, 0.83999997, 0.84375   ])

In [16]:
accuracies.mean(), accuracies.std()

(0.8511249959468842, 0.016728069665139542)

```python
# Affinage d'hyper-paramètres par validation croisée : GridSearchCV
from sklearn.model_selection import GridSearchCV
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [5, 25],
              'epochs': [100], #, 300
              'optimizer': ['adam']} #, 'rmsprop'

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 8)

grid_search = grid_search.fit(X_train, y_train)

grid_search.best_params_, grid_search.best_score_
```

## Sauvegarde et chargement des réseaux

Regarder les méthodes `save` et `load_model` de la librairie `keras.models` pour la sauvegarde et le chargement des modèle. Quel format de fichier utiliser ?

Si vous souhaitez ne sauvegarder que l'architecture du modèle (sans les poids ni la configuration d'entraînement), vous pouvez utiliser `to_json`.

Enfin, pour ne sauvegarder que les poids, vous avez la méthode `save_weights`.

In [17]:
mlp_clf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 84        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 133
Trainable params: 133
Non-trainable params: 0
_________________________________________________________________


### model.save()

Cette fonction sauvegarde:
- L'architecture du modèle, permettant de le recréer si nécessaire
- Les poids du modèles
- Les paramètres d'apprentissage (loss, optimizer, metrics dans l'étape `compile`).
- L'état de l'optimisation ce qui permet de reprendre l'apprentissage où on l'avait laissé

In [19]:
mlp_clf.save('models/mlp_clf_bankchurn.h5')

### model.load_model()

Cette fonction charge un modèle enregistré et l'ensemble de ses infos

In [20]:
from tensorflow.keras.models import load_model
new_model = load_model('models/mlp_clf_bankchurn.h5')

In [21]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 84        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 133
Trainable params: 133
Non-trainable params: 0
_________________________________________________________________


In [22]:
new_model.get_weights()

[array([[ 3.8806319e-01, -1.2984583e-01,  1.4795303e-01,  3.9133728e-01,
          5.0764030e-01,  5.8275330e-01],
        [ 1.9088547e-01,  7.5540558e-02, -3.6283594e-02,  2.4065277e-01,
          2.4200870e-01, -2.7432445e-01],
        [ 4.1927108e-01,  1.9702022e-01,  2.2537884e-01,  3.7907854e-01,
          1.9322513e-01,  4.8168102e-01],
        [ 4.0389273e-01, -4.3642536e-02, -6.1504757e-01,  4.2023441e-01,
          3.8203853e-01,  1.6709088e-01],
        [ 3.3347964e-01,  1.0324735e-03,  6.8761152e-01,  4.1838554e-01,
          4.5580164e-01,  5.1387954e-01],
        [ 1.2557963e-02,  3.4691364e-02,  2.6383467e-02,  2.7291201e-02,
          4.6784915e-02,  6.6219717e-02],
        [-7.7856427e-01,  5.8970433e-01, -4.0223137e-01, -8.5811847e-01,
         -8.5488600e-01,  1.4521357e-01],
        [-1.6854433e-02, -3.9606830e-03,  1.2825899e-01, -3.9928023e-02,
         -3.5306577e-02,  1.5700513e-01],
        [ 5.4187063e-02,  2.1316906e-02, -4.5991766e-01,  4.0319629e-02,
       

In [23]:
new_model.optimizer, new_model.loss

(<tensorflow.python.keras.optimizer_v2.adam.Adam at 0x7fb4b45c8350>,
 <function tensorflow.python.keras.losses.binary_crossentropy(y_true, y_pred, from_logits=False, label_smoothing=0)>)

### model.to_json()

Si on a juste besoin de sauvegarder la structure d'un réseau, sans ses poids ni ses paramètres d'entraînement, on peut utiliser un de ces deux fonctions.

In [24]:
# sauvegarder au format json
json_string = mlp_clf.to_json()
json_string

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "Dense", "config": {"name": "dense", "trainable": true, "batch_input_shape": [null, 13], "dtype": "float32", "units": 6, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "RandomUniform", "config": {"minval": -0.05, "maxval": 0.05, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 6, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "RandomUniform", "config": {"minval": -0.05, "maxval": 0.05, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_cons

In [25]:
# reconstruire un modèle depuis JSON
from tensorflow.keras.models import model_from_json
new_model2 = model_from_json(json_string)

In [26]:
new_model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 84        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 133
Trainable params: 133
Non-trainable params: 0
_________________________________________________________________


In [27]:
new_model2.get_weights()

[array([[ 0.01700081, -0.00347238,  0.02844789,  0.0320172 , -0.0301837 ,
         -0.03008308],
        [-0.03238676, -0.00604498, -0.02940136,  0.00465361,  0.00886982,
          0.0193612 ],
        [ 0.0264228 ,  0.04061887, -0.02440124,  0.03010553,  0.0001804 ,
          0.04316676],
        [-0.01615125, -0.04594085, -0.0428838 , -0.0362407 , -0.02900125,
         -0.02112159],
        [ 0.01483382, -0.03038022, -0.02934878, -0.00454221, -0.00641793,
         -0.03834749],
        [-0.02947449,  0.02840732,  0.0369069 ,  0.00082551, -0.04039915,
         -0.00138519],
        [ 0.01509402,  0.03247241,  0.04245781,  0.01226106, -0.02212368,
          0.03888113],
        [ 0.02721557,  0.04825569, -0.00094674,  0.02095871, -0.04847995,
         -0.02789102],
        [-0.01249887,  0.01965458,  0.04944919,  0.01430814, -0.02584171,
         -0.03950638],
        [ 0.01748535,  0.03361266,  0.03466997,  0.00238124,  0.02550166,
         -0.00499903],
        [-0.00866221,  0.03189

### model.save_weights()

Si jamais on veut uniquement les poids d'un modèle.

In [28]:
mlp_clf.save_weights('models/mes_poids.h5')

In [29]:
new_model3 = Sequential([
    Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 13),
    Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'),
    Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid')
])

In [30]:
new_model3.load_weights('models/mes_poids.h5')

In [31]:
new_model3.get_weights()

[array([[ 3.8806319e-01, -1.2984583e-01,  1.4795303e-01,  3.9133728e-01,
          5.0764030e-01,  5.8275330e-01],
        [ 1.9088547e-01,  7.5540558e-02, -3.6283594e-02,  2.4065277e-01,
          2.4200870e-01, -2.7432445e-01],
        [ 4.1927108e-01,  1.9702022e-01,  2.2537884e-01,  3.7907854e-01,
          1.9322513e-01,  4.8168102e-01],
        [ 4.0389273e-01, -4.3642536e-02, -6.1504757e-01,  4.2023441e-01,
          3.8203853e-01,  1.6709088e-01],
        [ 3.3347964e-01,  1.0324735e-03,  6.8761152e-01,  4.1838554e-01,
          4.5580164e-01,  5.1387954e-01],
        [ 1.2557963e-02,  3.4691364e-02,  2.6383467e-02,  2.7291201e-02,
          4.6784915e-02,  6.6219717e-02],
        [-7.7856427e-01,  5.8970433e-01, -4.0223137e-01, -8.5811847e-01,
         -8.5488600e-01,  1.4521357e-01],
        [-1.6854433e-02, -3.9606830e-03,  1.2825899e-01, -3.9928023e-02,
         -3.5306577e-02,  1.5700513e-01],
        [ 5.4187063e-02,  2.1316906e-02, -4.5991766e-01,  4.0319629e-02,
       

## Complément sur l'overfitting

Toujours sur les données de la banque, entrainer un réseau ayant une structure complexe avec beaucoup de neurones et de couches afin de générer une situation d'overfitting.  
Comparer l'accuracy sur les échantillons train et test pour confirmer le cas de sur-apprentissage.

Reprendre le même réseau en utilisant des layers `Dropout` pour réduire ce problème.  
Comparer à nouveau l'accuracy pour voir l'effet des `Dropout` sur l'overfitting.

Une autre méthode pour limiter le sur-apprentissage est la régularisation. Est-il possible d'en faire avec un réseau de neurones ? Si oui, allez-y

In [ ]:
# ANN - Situation d'overfitting
classifier = Sequential()
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', input_dim = 13))
classifier.add(Dense(units = 64, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

In [ ]:
score_train = classifier.evaluate(X_train, y_train) #0.9746
score_test = classifier.evaluate(X_test, y_test) #0.8135

In [ ]:
# ANN - avec Dropout
from keras.layers import Dropout

classifier = Sequential()
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', input_dim = 13))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 64, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

In [ ]:
score_train = classifier.evaluate(X_train, y_train) #0.9045
score_test = classifier.evaluate(X_test, y_test) #0.8465

In [ ]:
# ANN - avec régularisation l2
from keras import regularizers

classifier = Sequential()
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', input_dim = 13))
classifier.add(Dense(units = 64, kernel_initializer = 'uniform', activation = 'relu', kernel_regularizer=regularizers.l2(0.01)))
#ou kernel_regularizer="l2"
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', kernel_regularizer=regularizers.l2(0.01)))
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', kernel_regularizer=regularizers.l2(0.01)))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

In [ ]:
score_train = classifier.evaluate(X_train, y_train) #0.8696
score_test = classifier.evaluate(X_test, y_test) #0.8600